# 1) Import packages and data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from datetime import datetime
%matplotlib inline

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# get data from drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
df = pd.read_csv('Cocoon_official_15042023.csv')

In [ ]:
# assume `df` is your pandas dataframe containing "old_column_name_1", "old_column_name_2", and "old_column_name_3" columns
df.rename(columns={'Name': 'name', 'Price': 'price', 'Sales': 'sales','Rating':'rating','Rating record':'rating_record','Date':'date','Discount':'discount'}, inplace=True)

In [ ]:
# check if df reach needed non-null features and check for type of data for the next step
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3826 entries, 0 to 3825
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           3694 non-null   object
 1   price          3619 non-null   object
 2   sales          3661 non-null   object
 3   URL            3826 non-null   object
 4   rating         3586 non-null   object
 5   rating_record  3574 non-null   object
 6   date           3826 non-null   object
 7   discount       563 non-null    object
dtypes: object(8)
memory usage: 239.2+ KB


name              132
price             207
sales             165
URL                 0
rating            240
rating_record     252
date                0
discount         3263
dtype: int64

# 2) Remove unnecessary data

In [ ]:
# remove rows with missing values in the 'Name' column
df = df.dropna(subset=['name'])

Eliminate 'URL' column as I will not be needing it in the future. And As the "Price" in dataframe is "Price after discount", "Discount" is unnecessary, we remove "Discount" column as well.

In [ ]:
df=df.drop(columns=['URL','discount'])

# 3) Category and Product ID

In [ ]:
# define a function to assign categories based on product names
def assign_category(name):
    if isinstance(name, str) and "cà phê" in name.lower():
        return "Coffee"
    elif isinstance(name, str) and "hoa hồng" in name.lower():
        return "Rose"
    elif isinstance(name, str) and "bưởi" in name.lower():
        return "Pomelo"
    elif isinstance(name, str) and "nghệ" in name.lower():
        return "Ginger"
    elif isinstance(name, str) and "bí đao" in name.lower():
        return "Squash"
    else:
        return "Other"

# define a function to assign categories based on product names
def assign_ini(name):
    if isinstance(name, str) and "cà phê" in name.lower():
        return "C"
    elif isinstance(name, str) and "hoa hồng" in name.lower():
        return "R"
    elif isinstance(name, str) and "bưởi" in name.lower():
        return "P"
    elif isinstance(name, str) and "nghệ" in name.lower():
        return "G"
    elif isinstance(name, str) and "bí đao" in name.lower():
        return "S"
    else:
        return "O"

# apply the function to create a new "category" column
df["category"] = df["name"].apply(lambda name: assign_category(name))
df["ini_cat"] = df["name"].apply(lambda name: assign_ini(name))

In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat
0,HÀNG TẶNG KHÔNG BÁN - Gel rửa mặt cà phê Đắk L...,195,,NaN,NaN,2/9/2023,Coffee,C
1,Gel rửa mặt cà phê Đắk Lắk Cocoon cho làn da t...,195,Đã bán 175,4.9,64,2/9/2023,Coffee,C
2,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...,340,Đã bán 629,,,2/9/2023,Coffee,C
3,Combo Sáng Mịn Sạch Mụn: 1 Gel tắm bí đao Coco...,351,"Đã bán 1,4k",NaN,NaN,2/9/2023,Coffee,C
4,[Mã COSCN12 giảm 8%] Bơ dưỡng thể cà phê Đắk L...,215,"Đã bán 10,5k",5,"4,3k",2/9/2023,Coffee,C
...,...,...,...,...,...,...,...,...
3821,[Mã COSCN12 giảm 8%] Bigsize - Nước tẩy trang ...,₫250.750,"8,9k","4,9","3,2k",4/15/2023,Rose,R
3822,[Mã COSCN12 giảm 8%] Bigsize - Thạch hoa hồng ...,₫327.250,"6,9k","4,9","2,5k",4/15/2023,Rose,R
3823,[Mã COSCN12 giảm 8%] Nước hoa hồng (toner) Coc...,₫165.750,"13,1k",5,"4,8k",4/15/2023,Rose,R
3824,[Mã COSCN12 giảm 8%] Tinh chất hoa hồng (serum...,₫225.250,4k,5,"1,5k",4/15/2023,Rose,R


In [ ]:
#create seperate dataset for each category
coffee_df = df[df["category"] == "Coffee"]
rose_df = df[df["category"] == "Rose"]
pomelo_df = df[df["category"] == "Pomelo"]
ginger_df = df[df["category"] == "Ginger"]
squash_df = df[df["category"] == "Squash"]

# group by product name and assign a product ID to each category
coffee_df["group_ID"] = coffee_df.groupby("name").ngroup() + 1
rose_df["group_ID"] = rose_df.groupby("name").ngroup() + 1
pomelo_df["group_ID"] = pomelo_df.groupby("name").ngroup() + 1
ginger_df["group_ID"] = ginger_df.groupby("name").ngroup() + 1
squash_df["group_ID"] = squash_df.groupby("name").ngroup() + 1

In [ ]:
coffee_df['product_ID'] = coffee_df.apply(lambda row: str(row['ini_cat']) + str(row['group_ID']), axis=1)
rose_df['product_ID'] = rose_df.apply(lambda row: str(row['ini_cat']) + str(row['group_ID']), axis=1)
pomelo_df['product_ID'] = pomelo_df.apply(lambda row: str(row['ini_cat']) + str(row['group_ID']), axis=1)
ginger_df['product_ID'] = ginger_df.apply(lambda row: str(row['ini_cat']) + str(row['group_ID']), axis=1)
squash_df['product_ID'] = squash_df.apply(lambda row: str(row['ini_cat']) + str(row['group_ID']), axis=1)

In [ ]:
coffee_df = coffee_df.sort_values('group_ID').reset_index(drop=True)
rose_df = rose_df.sort_values('group_ID').reset_index(drop=True)
pomelo_df = pomelo_df.sort_values('group_ID').reset_index(drop=True)
ginger_df = ginger_df.sort_values('group_ID').reset_index(drop=True)
squash_df = squash_df.sort_values('group_ID').reset_index(drop=True)

In [ ]:
coffee_df.to_csv("Coffee_df.csv", index=True)
rose_df.to_csv("Rose_df.csv", index=True)
pomelo_df.to_csv("Pomelo_df.csv", index=True)
ginger_df.to_csv("Ginger_df.csv", index=True)
squash_df.to_csv("Squash_df.csv", index=True)

In [ ]:
coffee_df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID
0,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫295.000,125,"4,9",35,3/22/2023,Coffee,C,1,C1
1,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫295.000,83,"4,9",27,3/14/2023,Coffee,C,1,C1
2,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫295.000,138,"4,9",37,3/27/2023,Coffee,C,1,C1
3,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,97,"4,9",28,3/17/2023,Coffee,C,1,C1
4,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,37,5,12,3/9/2023,Coffee,C,1,C1
...,...,...,...,...,...,...,...,...,...,...
893,[Mã COSCN12 giảm 8%] Tẩy da chết mặt cà phê Đắ...,₫165.000,"63,6k","4,9","24,1k",30/03/2023,Coffee,C,22,C22
894,[Mã COSCN12 giảm 8%] Tẩy da chết mặt cà phê Đắ...,₫165.000,"62,3k","4,9","23,8k",3/13/2023,Coffee,C,22,C22
895,[Mã COSCN12 giảm 8%] Tẩy da chết mặt cà phê Đắ...,₫165.000,"63,5k","4,9","24,1k",29/03/2023,Coffee,C,22,C22
896,[Mã COSCN12 giảm 8%] Tẩy da chết mặt cà phê Đắ...,₫165.000,"62,4k","4,9","23,8k",3/14/2023,Coffee,C,22,C22


In [ ]:
# merge data with category type and ID back as official dataset 
df=pd.concat([coffee_df, rose_df, pomelo_df, ginger_df, squash_df], ignore_index=True)

# create Product_ID column based on category initial and category ID
df['group_ID'] = df.apply(lambda row: str(row['ini_cat']) + str(row['group_ID']), axis=1)

In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID
0,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫295.000,125,"4,9",35,3/22/2023,Coffee,C,C1,C1
1,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫295.000,83,"4,9",27,3/14/2023,Coffee,C,C1,C1
2,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫295.000,138,"4,9",37,3/27/2023,Coffee,C,C1,C1
3,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,97,"4,9",28,3/17/2023,Coffee,C,C1,C1
4,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,37,5,12,3/9/2023,Coffee,C,C1,C1
...,...,...,...,...,...,...,...,...,...,...
3689,[Mã COSCN12 giảm 8%] Xịt thâm mụn lưng bí đao ...,₫265.000,"1,3k","4,9",359,4/10/2023,Squash,S,S36,S36
3690,[Mã COSCN12 giảm 8%] Xịt thâm mụn lưng bí đao ...,₫225.250,"1,3k","4,9",349,4/3/2023,Squash,S,S36,S36
3691,[Mã COSCN12 giảm 8%] Xịt thâm mụn lưng bí đao ...,₫238.000,"1,3k","4,9",344,30/03/2023,Squash,S,S36,S36
3692,[Mã COSCN12 giảm 8%] Xịt thâm mụn lưng bí đao ...,₫265.000,"1,3k","4,9",361,4/11/2023,Squash,S,S36,S36


# 4) Dealing with missing value in Sales, Price, Rating and Rating record

The process of filling missing value in Sales, Price, Rating, Rating record can be accomblished by 2 steps:

**1) sort value by Product_ID and respective column (Sales, Price, Rating, Rating record):** Sorting the values in the DataFrame by 'Product ID' and 'Sales' is necessary to ensure that the fillna method fills NA values with the closest non-NA value within each group of products.The fillna method with the 'ffill' method parameter replaces missing values with the previous non-NA value in the same group. Therefore, if the DataFrame is not sorted by 'Product ID' and 'Sales', it may fill NA values with the wrong value from a different group.

**2) use Pandas' fillna method with the method parameter set to 'ffill' (forward fill)**
Forward fill is a method used to fill missing values in a Pandas DataFrame or Series. It involves filling the missing value with the last known or most recent value in the dataset. It is also known as "last observation carried forward" (LOCF) because the last valid observation is carried forward to fill in the missing values. The method works well when there is a temporal or sequential relationship between the data points.

In [ ]:
# filling missing value for Sales
df = df.sort_values(['product_ID', 'sales'])
df['sales'] = df.groupby('product_ID')['sales'].fillna(method='ffill')

# filling missing value for Price
df = df.sort_values(['product_ID', 'price'])
df['price'] = df.groupby('product_ID')['price'].fillna(method='ffill')

# filling missing value for Rating
df = df.sort_values(['product_ID', 'rating'])
df['rating'] = df.groupby('product_ID')['rating'].fillna(method='ffill')

# filling missing value for Rating record
df = df.sort_values(['product_ID', 'rating_record'])
df['rating_record'] = df.groupby('product_ID')['rating_record'].fillna(method='ffill')

In [ ]:
# check if df reach needed non-null features and check for type of data for the next step
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3694 entries, 4 to 3085
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           3694 non-null   object
 1   price          3694 non-null   object
 2   sales          3694 non-null   object
 3   rating         3694 non-null   object
 4   rating_record  3694 non-null   object
 5   date           3694 non-null   object
 6   category       3694 non-null   object
 7   ini_cat        3694 non-null   object
 8   group_ID       3694 non-null   object
 9   product_ID     3694 non-null   object
dtypes: object(10)
memory usage: 317.5+ KB


name             0
price            0
sales            0
rating           0
rating_record    0
date             0
category         0
ini_cat          0
group_ID         0
product_ID       0
dtype: int64

# 5) Date

In [ ]:
# filter dataframe into two subset right_dates and error_dates, which are date in the right format "mm/dd/yyyy" and those aren't 
e_dates = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce').isna()
error_dates = df[e_dates]

r_dates = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce').notna()
right_dates = df[r_dates]

#change date in wrong format to right format
error_dates['date'] = pd.to_datetime(error_dates['date'], format='%d/%m/%Y').dt.strftime('%m/%d/%Y')

#combine subset back to one original one
df = pd.concat([error_dates, right_dates], ignore_index=True)
# convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])
# sort date by ascending order:
df = df.sort_values(by='date')

In [ ]:
# reset the index
df = df.reset_index()

In [ ]:
df=df.drop(columns=['index'])

In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID
0,[Mã COSCN12 giảm 8%] Combo làm sạch da chết và...,323,"Đã bán 1,6k",5,576,2023-02-09,Coffee,C,C19,C19
1,HÀNG TẶNG KHÔNG BÁN - Bigsize - Nước nghệ Hưng...,₫295.000,"1,2k",5,407,2023-02-09,Ginger,G,G4,G4
2,Thạch nghệ Hưng Yên giúp làm sáng da và mờ thâ...,₫195.000,"2,6k",4.9,810,2023-02-09,Ginger,G,G15,G15
3,HÀNG TẶNG KHÔNG BÁN - Bigsize Mặt nạ nghệ Hưng...,₫345.000,"1,5k",5,646,2023-02-09,Ginger,G,G6,G6
4,Gel rửa mặt cà phê Đắk Lắk Cocoon cho làn da t...,195,Đã bán 175,4.9,64,2023-02-09,Coffee,C,C4,C4
...,...,...,...,...,...,...,...,...,...,...
3689,Bigsize - Thạch nghệ Hưng Yên giúp làm sáng da...,₫327.250,"1,2k","4,9",402,2023-04-15,Ginger,G,G3,G3
3690,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Nước bí đ...,₫340.000,396,"4,9",137,2023-04-15,Squash,S,S21,S21
3691,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Nước hoa ...,₫340.000,279,"4,9",81,2023-04-15,Rose,R,R7,R7
3692,Combo tóc dài bóng mượt Cocoon: 1 Nước dưỡng t...,₫266.050,"8,7k","4,9","3,4k",2023-04-15,Pomelo,P,P1,P1


# 6) Sales

4.1) Remove "Da ban" part in "Sales" and  to make "Sales" in the same structure.

In [ ]:
da_ban_data = df[df['sales'].str.startswith('Đã bán ')]
non_da_ban_data = df[~df['sales'].str.startswith('Đã bán ')]
# remove "Đã bán" part from beginning of each string
da_ban_data['sales'] = da_ban_data['sales'].str.lstrip('Đã bán ')
# merge filtered dataset back into original dataset
df = pd.concat([da_ban_data, non_da_ban_data], ignore_index=True)

4.2) turn "k" to thousands and change data type (to make "Sales" in the same type and structure)

In [ ]:
def convert_to_numeric(strings):
    if "k" in strings:
        num = float(strings.rstrip('k').replace(',', '.'))*1000
        return num
    else:
        return strings
df['sales'] = df['sales'].apply(convert_to_numeric)
df['sales'] = pd.to_numeric(df['sales'], errors='coerce')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3694 entries, 0 to 3693
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           3694 non-null   object        
 1   price          3694 non-null   object        
 2   sales          3668 non-null   float64       
 3   rating         3694 non-null   object        
 4   rating_record  3694 non-null   object        
 5   date           3694 non-null   datetime64[ns]
 6   category       3694 non-null   object        
 7   ini_cat        3694 non-null   object        
 8   group_ID       3694 non-null   object        
 9   product_ID     3694 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 288.7+ KB


In [ ]:
# filling missing value for Sales
df = df.sort_values(['product_ID', 'sales'])
df['sales'] = df.groupby('product_ID')['sales'].fillna(method='ffill')

In [ ]:
# there are 26 null values in sales after removing Đã bán and k in the entries, which means they were empty string
df.isnull().sum()

name             0
price            0
sales            0
rating           0
rating_record    0
date             0
category         0
ini_cat          0
group_ID         0
product_ID       0
dtype: int64

# 7) Rating and Rating record


In [ ]:
# replacing k with thousand in rating_record column
df['rating_record'] = df['rating_record'].apply(convert_to_numeric)
df['rating_record'] = pd.to_numeric(df['rating_record'], errors='coerce')

In [ ]:
# filling missing value for rating record
df = df.sort_values(['product_ID', 'rating_record'])
df['rating_record'] = df.groupby('product_ID')['rating_record'].fillna(method='ffill')

In [ ]:
# replacing no rating into 0
df['rating'] = df['rating'].str.replace('Chưa Có Đánh Giá', '0')
# trim whitespace from the "Rating" column
df['rating'] = df['rating'].str.strip()

In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID
989,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,25.0,5,9.0,2023-03-03,Coffee,C,C1,C1
1058,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,25.0,5,9.0,2023-03-04,Coffee,C,C1,C1
1083,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,25.0,5,9.0,2023-03-05,Coffee,C,C1,C1
1142,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,37.0,5,12.0,2023-03-06,Coffee,C,C1,C1
1241,Bigsize - Gel rửa mặt cà phê Đắk Lắk Cocoon ch...,₫250.750,37.0,5,12.0,2023-03-07,Coffee,C,C1,C1
...,...,...,...,...,...,...,...,...,...,...
2511,HÀNG TẶNG KHÔNG BÁN - Nước bí đao cân bằng da ...,₫195.000,2200.0,5,766.0,2023-03-29,Squash,S,S8,S8
2588,HÀNG TẶNG KHÔNG BÁN - Nước bí đao cân bằng da ...,₫195.000,2300.0,5,777.0,2023-03-30,Squash,S,S8,S8
2603,HÀNG TẶNG KHÔNG BÁN - Nước bí đao cân bằng da ...,₫195.000,2300.0,5,783.0,2023-03-31,Squash,S,S8,S8
3001,HÀNG TẶNG KHÔNG BÁN - Nước bí đao cân bằng da ...,₫195.000,2400.0,5,793.0,2023-04-05,Squash,S,S8,S8


In [ ]:
print(df['rating'].value_counts())

4,9    1792
5      1382
4.9     359
4,8      88
0        23
         19
4        11
4,7       7
4,5       5
4,6       5
4.7       2
4,3       1
Name: rating, dtype: int64


In [ ]:
df['rating'] = df['rating'].str.replace(',', '.')

In [ ]:
# filling missing value for rating 
df = df.sort_values(['product_ID', 'rating'])
df['rating'] = df.groupby('product_ID')['rating'].fillna(method='ffill')

In [ ]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

After replacing "Chưa có đánh giá" with "0" and replacing "," with ".". There are 19 null values in rating. Even after filling using method *ffill*, null values remains the same. Which means these mising value can only be removed from the model.

In [ ]:
df.dropna(subset=['rating'], inplace=True)

In [ ]:
df.isnull().sum()

name             0
price            0
sales            0
rating           0
rating_record    0
date             0
category         0
ini_cat          0
group_ID         0
product_ID       0
dtype: int64

# 8) Price

In [ ]:
# extract only the numeric part of the "Price" column
df["price"] = df["price"].str.extract(r"(\d+\.?\d*)").astype(float)

In [ ]:
df.dropna(subset=['price'], inplace=True)

In [ ]:
df = df.sort_values('date')
df = df.reset_index(drop=True)

In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID
0,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Nước nghệ...,340.00,409.0,4.9,137.0,2023-02-09,Ginger,G,G17,G17
1,Bigsize - Sữa rửa mặt nghệ Hưng Yên Cocoon giú...,295.00,2900.0,4.9,938.0,2023-02-09,Ginger,G,G2,G2
2,[Mã COSCN12 giảm 8%] Combo Cà phê Đắk Lắk làm ...,107.00,5300.0,5.0,2000.0,2023-02-09,Coffee,C,C18,C18
3,Bigsize - Nước nghệ Hưng Yên (toner) Cocoon là...,295.00,1100.0,5.0,304.0,2023-02-09,Ginger,G,G1,G1
4,[Mã COSCN12 giảm 8%] Bigsize - Mặt nạ nghệ Hưn...,345.00,4000.0,4.9,1600.0,2023-02-09,Ginger,G,G18,G18
...,...,...,...,...,...,...,...,...,...,...
3669,[Mã COSCN12 giảm 8%] Bigsize - Mặt nạ hoa hồng...,345.00,2100.0,5.0,786.0,2023-04-15,Rose,R,R8,R8
3670,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...,340.00,763.0,4.9,242.0,2023-04-15,Coffee,C,C16,C16
3671,[Mã COSCN12 giảm 8%] Bigsize - Nước tẩy trang ...,250.75,8900.0,4.9,3200.0,2023-04-15,Rose,R,R9,R9
3672,[Mã COSCN12 giảm 8%] Dầu tẩy trang hoa hồng Co...,153.00,5900.0,4.9,2000.0,2023-04-15,Rose,R,R15,R15


In [ ]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3674 entries, 0 to 3673
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           3674 non-null   object        
 1   price          3674 non-null   float64       
 2   sales          3674 non-null   float64       
 3   rating         3674 non-null   float64       
 4   rating_record  3674 non-null   float64       
 5   date           3674 non-null   datetime64[ns]
 6   category       3674 non-null   object        
 7   ini_cat        3674 non-null   object        
 8   group_ID       3674 non-null   object        
 9   product_ID     3674 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 287.2+ KB


name             0
price            0
sales            0
rating           0
rating_record    0
date             0
category         0
ini_cat          0
group_ID         0
product_ID       0
dtype: int64

# 9) Date in a week

In [ ]:
# create a new column with the day of the week
df['day_of_week'] = df['date'].dt.day_name()

# group the sales data by day of the week and calculate the total sales
sales_by_day = df.groupby('day_of_week')['sales'].sum()

sales_by_day

day_of_week
Friday       5642643.0
Monday       4124323.0
Saturday     4933623.0
Sunday       4208156.0
Thursday     4708250.0
Tuesday      4225970.0
Wednesday    4506449.0
Name: sales, dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3674 entries, 0 to 3673
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           3674 non-null   object        
 1   price          3674 non-null   float64       
 2   sales          3674 non-null   float64       
 3   rating         3674 non-null   float64       
 4   rating_record  3674 non-null   float64       
 5   date           3674 non-null   datetime64[ns]
 6   category       3674 non-null   object        
 7   ini_cat        3674 non-null   object        
 8   group_ID       3674 non-null   object        
 9   product_ID     3674 non-null   object        
 10  day_of_week    3674 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 315.9+ KB


# 10) Arange data by date in ascending order and reset index

In [ ]:
df = df.sort_values('date')
df = df.reset_index(drop=True)

In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID,day_of_week
0,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Nước nghệ...,340.00,409.0,4.9,137.0,2023-02-09,Ginger,G,G17,G17,Thursday
1,Combo Sáng Mịn Sạch Mụn: 1 Gel tắm bí đao Coco...,351.00,1400.0,4.9,490.0,2023-02-09,Coffee,C,C3,C3,Thursday
2,HÀNG TẶNG KHÔNG BÁN - Mặt nạ nghệ Hưng Yên Coc...,145.00,6200.0,5.0,2000.0,2023-02-09,Ginger,G,G7,G7,Thursday
3,Gel rửa mặt cà phê Đắk Lắk Cocoon cho làn da t...,195.00,175.0,4.9,64.0,2023-02-09,Coffee,C,C4,C4,Thursday
4,[Mã COSCN12 giảm 8%] Tẩy da chết môi cà phê Đắ...,75.00,20100.0,5.0,8100.0,2023-02-09,Coffee,C,C21,C21,Thursday
...,...,...,...,...,...,...,...,...,...,...,...
3669,SPECIAL COMBO Làm Sạch Cho Da Dầu Mụn Từ Bí Đa...,295.00,2500.0,5.0,1000.0,2023-04-15,Squash,S,S13,S13,Saturday
3670,[COMBO MUA 1 TẶNG 1 ÁP DỤNG 01/04-30/04] Kem c...,395.00,621.0,4.9,172.0,2023-04-15,Squash,S,S20,S20,Saturday
3671,[Mã COSCN12 giảm 8%] Sữa rửa mặt nghệ Hưng Yên...,165.75,10000.0,4.9,3600.0,2023-04-15,Ginger,G,G23,G23,Saturday
3672,SPECIAL COMBO Gel rửa mặt cà phê 140ml+Tẩy da ...,360.00,99.0,5.0,27.0,2023-04-15,Coffee,C,C10,C10,Saturday


In [ ]:
# count unique names
unique_names = df['name'].nunique()

print(unique_names)

119


In [ ]:
df=df.drop(columns=['name'])

In [ ]:
# Write the DataFrame to a CSV file
df.to_csv("Preprocessed_data.csv", index=True)

# 11) Check data information for final result


In [ ]:
df1= pd.read_csv('Preprocessed_data.csv')
df1

,Unnamed: 0,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID,day_of_week
0,0,340.00,409.0,4.9,137.0,2023-02-09,Ginger,G,G17,G17,Thursday
1,1,351.00,1400.0,4.9,490.0,2023-02-09,Coffee,C,C3,C3,Thursday
2,2,145.00,6200.0,5.0,2000.0,2023-02-09,Ginger,G,G7,G7,Thursday
3,3,195.00,175.0,4.9,64.0,2023-02-09,Coffee,C,C4,C4,Thursday
4,4,75.00,20100.0,5.0,8100.0,2023-02-09,Coffee,C,C21,C21,Thursday
...,...,...,...,...,...,...,...,...,...,...,...
3669,3669,295.00,2500.0,5.0,1000.0,2023-04-15,Squash,S,S13,S13,Saturday
3670,3670,395.00,621.0,4.9,172.0,2023-04-15,Squash,S,S20,S20,Saturday
3671,3671,165.75,10000.0,4.9,3600.0,2023-04-15,Ginger,G,G23,G23,Saturday
3672,3672,360.00,99.0,5.0,27.0,2023-04-15,Coffee,C,C10,C10,Saturday


In [ ]:
df1=df1.drop(columns=['Unnamed: 0'])

In [ ]:
df1

,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID,day_of_week
0,340.00,409.0,4.9,137.0,2023-02-09,Ginger,G,G17,G17,Thursday
1,351.00,1400.0,4.9,490.0,2023-02-09,Coffee,C,C3,C3,Thursday
2,145.00,6200.0,5.0,2000.0,2023-02-09,Ginger,G,G7,G7,Thursday
3,195.00,175.0,4.9,64.0,2023-02-09,Coffee,C,C4,C4,Thursday
4,75.00,20100.0,5.0,8100.0,2023-02-09,Coffee,C,C21,C21,Thursday
...,...,...,...,...,...,...,...,...,...,...
3669,295.00,2500.0,5.0,1000.0,2023-04-15,Squash,S,S13,S13,Saturday
3670,395.00,621.0,4.9,172.0,2023-04-15,Squash,S,S20,S20,Saturday
3671,165.75,10000.0,4.9,3600.0,2023-04-15,Ginger,G,G23,G23,Saturday
3672,360.00,99.0,5.0,27.0,2023-04-15,Coffee,C,C10,C10,Saturday


In [ ]:
#print(df.describe())
df1.describe()

,price,sales,rating,rating_record
count,3674.000000,3674.000000,3674.000000,3674.000000
mean,290.052912,8804.957539,4.900245,3402.378334
std,147.988882,18032.597284,0.396627,7472.039565
min,60.000000,0.000000,0.000000,0.000000
25%,195.000000,824.000000,4.900000,268.250000
50%,290.000000,3900.000000,4.900000,1300.000000
75%,345.000000,9600.000000,5.000000,3500.000000
max,955.000000,126300.000000,5.000000,124800.000000


In [ ]:
# specify a formatting function that converts the scientific notation to a fixed-point notation
pd.options.display.float_format = '{:.2f}'.format
# Rename the columns to include count, std, and mean
stats = df1.describe()
#save result of statiscal dataset information into csv file 
stats.to_csv("Statiscal result.csv", index= True)
stats

,price,sales,rating,rating_record
count,3674.00,3674.00,3674.00,3674.00
mean,290.05,8804.96,4.90,3402.38
std,147.99,18032.60,0.40,7472.04
min,60.00,0.00,0.00,0.00
25%,195.00,824.00,4.90,268.25
50%,290.00,3900.00,4.90,1300.00
75%,345.00,9600.00,5.00,3500.00
max,955.00,126300.00,5.00,124800.00


# 12) Facebook preprocessing

In [ ]:
fb = pd.read_csv('Fb_020304.csv')

In [ ]:
fb

,post_date,number_of_likes,number_of_comments,number_of_shares,number_of_comment_by_Cocoon,comment_with_tags,character,hashtag_count
0,2023-02-01,15000.0,59,41,10,16,1932,5
1,2023-02-02,13000.0,59,15,5,8,798,5
2,2023-02-03,77.0,3,0,1,0,1062,4
3,2023-02-05,2400.0,15,5,4,2,1898,5
4,2023-02-07,430.0,26,2,5,4,671,5
...,...,...,...,...,...,...,...,...
60,2023-04-06,6600.0,352,79,29,37,436,6
61,2023-04-12,2000.0,13,3,3,1,1377,6
62,2023-04-13,120.0,22,3,8,2,372,4
63,2023-04-14,3600.0,60,59,6,14,1259,4


In [ ]:
#change data type into datetime
fb['post_date'] = fb['post_date'].apply(lambda x: datetime.strptime(x, '%Y/%m/%d'))

In [ ]:
# Group the DataFrame by 'post_date'
date_counts = fb['post_date'].value_counts()

# Filter the results to only show the dates that occur more than once
duplicate_dates = date_counts[date_counts > 1].index

# Select the rows with duplicate dates
duplicate_entries = fb[fb['post_date'].isin(duplicate_dates)]

In [ ]:
duplicate_entries

,post_date,post_type,aim,post_category,content,hashtags,number_of_likes,number_of_comments,number_of_shares,number_of_comment_by_Cocoon,comment_with_tags
31,2023-03-06,Picture,Product promo,Pomelo,CHÍNH THỨC RA MẮT: KEM Ủ TÓC BƯỞI\r\nGiải pháp...,#Cocoon #CocoonVietnam #Myphamthuanchay #Vegan...,"9,5k",159,47,23,20
32,2023-03-06,Picture,Product promo,Pomelo,[CHÍNH THỨC MỞ BÁN] \r\nKEM Ủ TÓC BƯỞI - HỒI S...,#Cocoon #CocoonVietnam #Vegancosmetics #Mypham...,"5,1k",71,19,18,20


In [ ]:
def convert_to_numeric(strings):
    if "k" in strings:
        num = float(strings.rstrip('k').replace(',', '.'))*1000
        return num
    else:
        return strings

In [ ]:
# run fuction "convert_to_numeric" and apply to fb dataframe as follow 
fb['number_of_likes']= fb['number_of_likes'].apply(convert_to_numeric)
fb['number_of_likes'] = pd.to_numeric(fb['number_of_likes'], errors='coerce').fillna(0)

In [ ]:
# count length of data in "Content" column
fb['character'] = fb['content'].str.len()

In this code, the .apply() function is used with a lambda function that splits each string in the "hashtags" column by "#" and counts the number of resulting strings (which is equal to the number of hashtag words). The - 1 is used to account for the fact that the first string in the split result will not be a hashtag word. The resulting count is then assigned to a new column "num_hashtags".

In [ ]:
hashtags = fb['hashtags'].astype(str)  # convert the column to string data type
fb['hashtag_count'] = hashtags.apply(lambda x: len(x.split('#')) - 1)  # count the number of hashtags

In [ ]:
fb=fb.drop(columns=['content','hashtags'])

In [ ]:
# import category encoders
!pip install category_encoders
import category_encoders as ce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
# encode categorical variables with ordinal encoding

encoder = ce.OrdinalEncoder(cols=['post_type', 'aim', 'post_category'])

In [ ]:
fb.columns

Index(['post_date', 'number_of_likes', 'number_of_comments',
       'number_of_shares', 'number_of_comment_by_Cocoon', 'comment_with_tags',
       'character', 'hashtag_count'],
      dtype='object')

In [ ]:
fb = fb.groupby('post_date')[['post_type', 'aim', 'post_category', 'number_of_likes',
       'number_of_comments', 'number_of_shares', 'number_of_comment_by_Cocoon',
       'comment_with_tags', 'character', 'hashtag_count']].sum().reset_index()

In [ ]:
# Write the DataFrame to a CSV file
fb.to_csv("Preprocessed_fb.csv", index=True)

In [ ]:
video_df= fb[(fb['post_type'] == 'Video')]
picture_pf= fb[(fb['post_type'] == 'Picture')]

KeyError: ignored

In [ ]:
# specify a formatting function that converts the scientific notation to a fixed-point notation
pd.options.display.float_format = '{:.2f}'.format
# Rename the columns to include count, std, and mean
vid_stats = video_df.describe()
pic_stats = picture_pf.describe()

In [ ]:
pic_stats

In [ ]:
vid_stats

In [ ]:
for aim_value in fb['aim'].unique():
    aim_df = fb[fb['aim'] == aim_value]

In [ ]:
aim_df

# 13) Merge Facebook data and Shopee data 

In [ ]:
fb

,post_date,number_of_likes,number_of_comments,number_of_shares,number_of_comment_by_Cocoon,comment_with_tags,character,hashtag_count
0,2023-02-01,15000.00,59,41,10,16,1932,5
1,2023-02-02,13000.00,59,15,5,8,798,5
2,2023-02-03,77.00,3,0,1,0,1062,4
3,2023-02-05,2400.00,15,5,4,2,1898,5
4,2023-02-07,430.00,26,2,5,4,671,5
...,...,...,...,...,...,...,...,...
60,2023-04-06,6600.00,352,79,29,37,436,6
61,2023-04-12,2000.00,13,3,3,1,1377,6
62,2023-04-13,120.00,22,3,8,2,372,4
63,2023-04-14,3600.00,60,59,6,14,1259,4


In [ ]:
df

,name,price,sales,rating,rating_record,date,category,ini_cat,group_ID,product_ID,day_of_week
0,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Nước nghệ...,340.00,409.0,4.9,137.0,2023-02-09,Ginger,G,G17,G17,Thursday
1,Bigsize - Sữa rửa mặt nghệ Hưng Yên Cocoon giú...,295.00,2900.0,4.9,938.0,2023-02-09,Ginger,G,G2,G2,Thursday
2,[Mã COSCN12 giảm 8%] Combo Cà phê Đắk Lắk làm ...,107.00,5300.0,5.0,2000.0,2023-02-09,Coffee,C,C18,C18,Thursday
3,Bigsize - Nước nghệ Hưng Yên (toner) Cocoon là...,295.00,1100.0,5.0,304.0,2023-02-09,Ginger,G,G1,G1,Thursday
4,[Mã COSCN12 giảm 8%] Bigsize - Mặt nạ nghệ Hưn...,345.00,4000.0,4.9,1600.0,2023-02-09,Ginger,G,G18,G18,Thursday
...,...,...,...,...,...,...,...,...,...,...,...
3669,[Mã COSCN12 giảm 8%] Bigsize - Mặt nạ hoa hồng...,345.00,2100.0,5.0,786.0,2023-04-15,Rose,R,R8,R8,Saturday
3670,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...,340.00,763.0,4.9,242.0,2023-04-15,Coffee,C,C16,C16,Saturday
3671,[Mã COSCN12 giảm 8%] Bigsize - Nước tẩy trang ...,250.75,8900.0,4.9,3200.0,2023-04-15,Rose,R,R9,R9,Saturday
3672,[Mã COSCN12 giảm 8%] Dầu tẩy trang hoa hồng Co...,153.00,5900.0,4.9,2000.0,2023-04-15,Rose,R,R15,R15,Saturday


In [ ]:
daily_sales = df.groupby('date')[['sales', 'rating', 'rating_record']].sum().reset_index()

In [ ]:
min_sales_row = daily_sales[daily_sales['sales'] == 166842]
print(min_sales_row)

         date     sales  rating  rating_record
30 2023-03-11  166842.0   113.0        63351.0


In [ ]:
next_to_min= daily_sales[daily_sales['date'] == '2023-04-12']
print(next_to_min)

         date     sales  rating  rating_record
62 2023-04-12  584115.0   345.0       214432.0


In [ ]:
max_sales_row = daily_sales[daily_sales['sales'] == 938750]
print(max_sales_row)

         date     sales  rating  rating_record
64 2023-04-14  938750.0   680.8       342819.0


In [ ]:
daily_sales

,date,sales,rating,rating_record
0,2023-02-09,265501.0,133.7,97292.0
1,2023-02-10,280093.0,163.0,165439.0
2,2023-02-11,428111.0,163.2,156945.0
3,2023-02-12,443558.0,177.9,165842.0
4,2023-02-13,328475.0,177.9,116791.0
...,...,...,...,...
61,2023-04-11,568700.0,330.1,208510.0
62,2023-04-12,584115.0,345.0,214432.0
63,2023-04-13,471140.0,251.7,172783.0
64,2023-04-14,938750.0,680.8,342819.0


In [ ]:
daily_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           66 non-null     datetime64[ns]
 1   sales          66 non-null     float64       
 2   rating         66 non-null     float64       
 3   rating_record  66 non-null     float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 2.2 KB


In [ ]:
# Create a DataFrame with a date range
start_date = '2023-02-09'
end_date = '2023-04-15'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
date_df = pd.DataFrame({'post_date': date_range})
date_sales = pd.DataFrame({'date': date_range})

# Merge the new DataFrame with your existing DataFrame
merged_fb = pd.merge(date_df, fb, on='post_date', how='left')
merged_sales = pd.merge(date_sales, daily_sales, on='date', how='left')
# Replace NaN values with 0
merged_fb.fillna(0, inplace=True)
merged_sales.fillna(0, inplace=True)
final_merged= pd.concat([merged_fb, merged_sales], axis=1)

In [ ]:
final_merged

,post_date,number_of_likes,number_of_comments,number_of_shares,number_of_comment_by_Cocoon,comment_with_tags,character,hashtag_count,date,sales,rating,rating_record
0,2023-02-09,110.0,5.0,2.0,1.0,0.0,340.0,4.0,2023-02-09,265501.0,133.7,97292.0
1,2023-02-10,3400.0,14.0,12.0,0.0,3.0,1618.0,5.0,2023-02-10,280093.0,163.0,165439.0
2,2023-02-11,216.0,77.0,3.0,32.0,4.0,944.0,4.0,2023-02-11,428111.0,163.2,156945.0
3,2023-02-12,10000.0,232.0,60.0,38.0,200.0,2267.0,4.0,2023-02-12,443558.0,177.9,165842.0
4,2023-02-13,91.0,3.0,4.0,0.0,2.0,408.0,4.0,2023-02-13,328475.0,177.9,116791.0
...,...,...,...,...,...,...,...,...,...,...,...,...
61,2023-04-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-04-11,568700.0,330.1,208510.0
62,2023-04-12,2000.0,13.0,3.0,3.0,1.0,1377.0,6.0,2023-04-12,584115.0,345.0,214432.0
63,2023-04-13,120.0,22.0,3.0,8.0,2.0,372.0,4.0,2023-04-13,471140.0,251.7,172783.0
64,2023-04-14,3600.0,60.0,59.0,6.0,14.0,1259.0,4.0,2023-04-14,938750.0,680.8,342819.0


In [ ]:
# Write the DataFrame to a CSV file
final_merged.to_csv("Final_data.csv", index=True)